# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Output File (CSV)
output_data_file = "../Resources/cities.csv"


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Importing csv file -- for pick up later tasks
weather_data = pd.read_csv(output_data_file)
weather_data

,City,Country,Date,Latitude,Longitude,Temp,Humidity,Max Temp,Cloudiness,Wind Speed
0,Yellowknife,CA,1579738296,62.46,-114.35,5.00,84,5.00,90,10.29
1,Kununurra,AU,1579738841,-15.77,128.73,84.20,74,84.20,85,9.35
2,Chokurdakh,RU,1579738308,70.63,147.92,-20.96,97,-20.96,100,8.68
3,Patnongon,PH,1579738841,10.91,121.99,77.36,69,77.36,0,6.82
4,Jamestown,US,1579738202,42.10,-79.24,26.38,53,32.00,1,9.17
...,...,...,...,...,...,...,...,...,...,...
568,Naze,JP,1579739685,28.37,129.48,70.75,78,71.60,75,20.80
569,Sokoni,TZ,1579739685,-6.43,39.55,81.61,83,81.61,100,9.66
570,Vostok,RU,1579739685,46.49,135.88,0.12,90,0.12,43,4.18
571,Namie,JP,1579739685,37.48,141.00,39.33,93,45.00,75,2.24


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
coordinates = weather_data[["Latitude", "Longitude"]]

# Humidity numbers for heat map weight
humidity = weather_data["Humidity"]


In [45]:
# Finding max humidity for intensity
max_humid = weather_data["Humidity"].max()

# Setting figure size
figure_layout = {
    'width': '900px',
    'height': '550px'
}

# Plot heatmap
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=max_humid,
                                point_radius=5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='550px', width='900px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
weather = weather_data.loc[(weather_data["Max Temp"] > 60) & (weather_data["Max Temp"] < 75)]
weather = weather.loc[(weather["Humidity"] > 40) & (weather["Humidity"] < 60)]
weather = weather.loc[weather["Wind Speed"] < 10]
weather


,City,Country,Date,Latitude,Longitude,Temp,Humidity,Max Temp,Cloudiness,Wind Speed
11,Los Llanos de Aridane,ES,1579738841,28.66,-17.92,59.07,51,60.80,20,8.05
29,Busselton,AU,1579738565,-33.65,115.33,67.21,44,71.01,76,9.75
100,Pittsburg,US,1579738678,38.03,-121.88,59.36,55,63.00,1,3.83
143,Keetmanshoop,NaN,1579739042,-26.58,18.13,69.80,43,69.80,0,9.17
151,Cape Town,ZA,1579738802,-33.93,18.42,66.00,59,66.99,0,6.93
153,Opuwo,NaN,1579739043,-18.06,13.84,71.58,42,71.58,56,3.27
276,Bahia Honda,CU,1579739252,22.90,-83.16,60.85,59,60.85,40,9.89
408,Paarl,ZA,1579739398,-33.73,18.98,65.98,59,66.99,0,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store into variable named hotel_df
hotel_df = weather.copy()

# Add hotel name column
hotel_df["Hotel Name"] = ""
hotel_df


,City,Country,Date,Latitude,Longitude,Temp,Humidity,Max Temp,Cloudiness,Wind Speed,Hotel Name
11,Los Llanos de Aridane,ES,1579738841,28.66,-17.92,59.07,51,60.80,20,8.05,
29,Busselton,AU,1579738565,-33.65,115.33,67.21,44,71.01,76,9.75,
100,Pittsburg,US,1579738678,38.03,-121.88,59.36,55,63.00,1,3.83,
143,Keetmanshoop,NaN,1579739042,-26.58,18.13,69.80,43,69.80,0,9.17,
151,Cape Town,ZA,1579738802,-33.93,18.42,66.00,59,66.99,0,6.93,
153,Opuwo,NaN,1579739043,-18.06,13.84,71.58,42,71.58,56,3.27,
276,Bahia Honda,CU,1579739252,22.90,-83.16,60.85,59,60.85,40,9.89,
408,Paarl,ZA,1579739398,-33.73,18.98,65.98,59,66.99,0,6.93,


In [16]:
# Base url - using nearby search
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Setting type to search
target_search = "hotel"

# Setting parameters
params = {"key": g_key,
         "radius": 5000,
         "keyword": target_search}

# Loop through the data frame
for index, row in hotel_df.iterrows():
    
    # get lat and lng
    Lat = row['Latitude']
    Lng = row['Longitude']
    
    # Update coordinates by key
    params['location'] = f"{Lat},{Lng}"
    
    # Make the request
    response = requests.get(base_url, params=params)
    hotel = response.json()
    
    # Extract results
    #vacay_hotels = hotel["results"]
    
    hotel_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]
    #print(response.url)
    

In [17]:
hotel_df

,City,Country,Date,Latitude,Longitude,Temp,Humidity,Max Temp,Cloudiness,Wind Speed,Hotel Name
11,Los Llanos de Aridane,ES,1579738841,28.66,-17.92,59.07,51,60.80,20,8.05,Residencial Las Alas S.L.
29,Busselton,AU,1579738565,-33.65,115.33,67.21,44,71.01,76,9.75,Esplanade Hotel
100,Pittsburg,US,1579738678,38.03,-121.88,59.36,55,63.00,1,3.83,Hampton Inn & Suites Pittsburg
143,Keetmanshoop,NaN,1579739042,-26.58,18.13,69.80,43,69.80,0,9.17,Quiver Inn Guesthouse
151,Cape Town,ZA,1579738802,-33.93,18.42,66.00,59,66.99,0,6.93,"Radisson Blu Hotel Waterfront, Cape Town"
153,Opuwo,NaN,1579739043,-18.06,13.84,71.58,42,71.58,56,3.27,Opuwo Country Lodge
276,Bahia Honda,CU,1579739252,22.90,-83.16,60.85,59,60.85,40,9.89,Casa Gualberto - Deysi
408,Paarl,ZA,1579739398,-33.73,18.98,65.98,59,66.99,0,6.93,Grande Roche Hotel


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [46]:
# Add marker layer ontop of heat map
figure_layout = {
    'width': '900px',
    'height': '550px'
}

fig = gmaps.figure(layout=figure_layout)

fig.add_layer(heat_layer)

hotel_layer = gmaps.marker_layer(locations,  
                                info_box_content=hotel_info)
fig.add_layer(hotel_layer)

# Display Map
fig



Figure(layout=FigureLayout(height='550px', width='900px'))